# Today, we will be talking about Pandas
**Note**: content adapted from [here](https://github.com/arunabh15091989/LighthouseLabs-LectureNotes/tree/master/W2D2)

## What is it?
- A Python package for working with **tables** 
- Similar to Excel and SQL
    - Faster (exists in memory on your computer)
    - More features to manipiulate, transform, and aggregate data
- Intuitive API (imperative, squential statements)
- Seamless integration with other Python packages (e.g. Machine Learning packages)
- Seamless integration with other data formats (.csv, SQL, HDF5, standard Python data types)

# Motivation
Think of how you would represent a table with column names in Python normally. Perhaps...

### A dictionary of lists
- Columns are keys
- Rows are index in the list

In [91]:
students = {
    'student_id': [1412, 94, 9351, 14],
    'name': ['Bonnie', 'Eric', 'Lynxi', 'Gad'],
    'course_mark': [82, 100, 12, 76],
    'species': ['human', 'human', 'cat', 'human']
}

### What are some operations we might want to do on this data?
1. Select a subset of columns
2. Filter out some rows based on an attribute
3. Group by some attribute
4. Compute some aggregate values within groups
5. Save to a file

How about we try out one of these to see how easy it is.

In [92]:
# Return a table with the mean course mark per-species

species_sums = {}
species_counts = {}
unique_species = set(students['species'])
for i in range(len(students['species'])):
    species = students['species'][i]
    course_mark = students['course_mark'][i]
    if species not in species_sums:
        species_sums[species] = 0
        species_counts[species] = 0
    species_sums[species] += course_mark
    species_counts[species] += 1

species_means = {}
for species in species_sums:
    species_means[species] = species_sums[species] / species_counts[species]

species_means

{'human': 86.0, 'cat': 12.0}

### Doing it in Pandas
Was the above fun? Do you find the code elegant? Easy to understand and write?

Luckily, the Pandas package exists to make our lives easier. Let's write the exact same code,
but using Pandas.

In [93]:
# Return a table with the mean course mark per-species

import pandas as pd        # To not have to write 'pandas' in full every time

students = pd.DataFrame(students)    # Can take in a dictionary of lists to instantiate a DataFrame
students

,student_id,name,course_mark,species
0,1412,Bonnie,82,human
1,94,Eric,100,human
2,9351,Lynxi,12,cat
3,14,Gad,76,human


In [94]:
species_means = students[['species', 'course_mark']].groupby('species').mean()
species_means

,course_mark
species,
cat,12
human,86


### Let's dissect the above a bit more

In [95]:
# Step 1: Filter out the columns we want to keep
students_filtered = students[['species', 'course_mark']]
students_filtered

,species,course_mark
0,human,82
1,human,100
2,cat,12
3,human,76


In [96]:
## Step 2: Group by the species column
students_grouped_by_species = students_filtered.groupby('species')
students_grouped_by_species

In [97]:
# Step 3: Specify how to aggregate the course_mark column
species_means = students_grouped_by_species.mean()
species_means

,course_mark
species,
cat,12
human,86


### As shown, Pandas makes use of **vectorized** operations
- Rather than use for-loops, we specify the operation that will apply to the structure as a whole (i.e. all the rows)
- By vectorizing, the code becomes more concise and more readable
- Pandas is optimized for vectorized operations (parallel vs. serial computation), which makes them **much faster**
- It is almost always possible to vectorize operations on Pandas data types

# The basics of Pandas

### Data structures
There are two data types of interest:
- Series (column)
- DataFrame (table)

In [98]:
# One way to construct a Series
series = pd.Series([10, 8, 62, 3], index=[94, 9351, 14, 1412])   # Optionally, can pass 'index' (row labels). Default will be 0, 1, 2, etc.

In [99]:
# One way to construct a DataFrame
df = pd.DataFrame({
    'name': ['Bonnie', 'Eric', 'Lynxi', 'Gad'],
    'course_mark': [82, 100, 12, 76],
    'species': ['human', 'human', 'cat', 'human']},
    index=[1412, 94, 9351, 14])

### Index-aligned vectorized operations
When you perform vectorized computations on 2 Pandas data structures, they are
automatically aligned by their "index"

In [100]:
df['course_mark'] + series        # '14' sums with '14', '94' sums with '94', etc. regardless of their order

14      138
94      110
1412     85
9351     20
dtype: int64

### Naming indices and columns
Sometimes, it is nice to name our columns so that we know what they represent.

In [101]:
series = series.rename('bonus')
series.index = series.index.rename('student_id')
series

student_id
94      10
9351     8
14      62
1412     3
Name: bonus, dtype: int64

In [102]:
df.index = df.index.rename('student_id')
df_renamed = df.rename(columns={'name': 'Name', 'course_mark': 'Course Mark', 'species': 'Species'})
df_renamed

,Name,Course Mark,Species
student_id,,,
1412,Bonnie,82,human
94,Eric,100,human
9351,Lynxi,12,cat
14,Gad,76,human


### Filtering columns (i.e. SQL SELECT)

In [103]:
# Select a single column
df['name']

student_id
1412    Bonnie
94        Eric
9351     Lynxi
14         Gad
Name: name, dtype: object

In [116]:
# Select multiple columns (and reordering them however you want)
df[['course_mark', 'name']]   # Make sure to pass in a list, otherwise it will look for a single column who's value is the tuple you pass in!

,course_mark,name
student_id,,
1412,82,Bonnie
94,100,Eric
9351,12,Lynxi
14,76,Gad


### Filtering rows (i.e. SQL WHERE)

In [112]:
# Select a single row by its position
df.iloc[1]

name            Eric
course_mark      100
species        human
Name: 94, dtype: object

In [113]:
# Select multiple rows by their position
df.iloc[1:3]

,name,course_mark,species
student_id,,,
94,Eric,100,human
9351,Lynxi,12,cat


In [114]:
# Select a single row by its index
df.loc[94]

name            Eric
course_mark      100
species        human
Name: 94, dtype: object

In [115]:
# Select multiple rows by their index (and reordering them however you want)
df.loc[[14, 1412]]

,name,course_mark,species
student_id,,,
14,Gad,76,human
1412,Bonnie,82,human


In [121]:
# Filtering rows by truth values
passing_mark = 60
passed_the_course = df['course_mark'] >= passing_mark
print(passed_the_course)    # Series of True/False values. Can use as a 'mask' to filter rows

# Only selects rows where 'passed_the_course' is True
df_passed = df[passed_the_course]    

# Typically, we would shorten the above statements to a single line
df_passed = df[df['course_mark'] >= passing_mark]
df_passed

student_id
1412     True
94       True
9351    False
14       True
Name: course_mark, dtype: bool


,name,course_mark,species
student_id,,,
1412,Bonnie,82,human
94,Eric,100,human
14,Gad,76,human


### Sorting (i.e. SQL ORDER BY)

In [122]:
# Sort rows by their index values
df.sort_index()

,name,course_mark,species
student_id,,,
14,Gad,76,human
94,Eric,100,human
1412,Bonnie,82,human
9351,Lynxi,12,cat


In [125]:
# Sort rows by a column's value
df.sort_values(by='name')

,name,course_mark,species
student_id,,,
1412,Bonnie,82,human
94,Eric,100,human
14,Gad,76,human
9351,Lynxi,12,cat


In [138]:
# Sort rows by multiple column values
df.sort_values(by=['species', 'name'], ascending=False)    # Sorts by 'species' first, then 'name' within the 'species' groups

,name,course_mark,species
student_id,,,
14,Gad,76,human
94,Eric,100,human
1412,Bonnie,82,human
9351,Lynxi,12,cat


### Grouping (i.e. SQL GROUP BY)

In [130]:
# Compute the mean course mark within each 'species'
df[['species', 'course_mark']].groupby('species').mean()

,course_mark
species,
cat,12
human,86


In [134]:
# Group by multiple columns
df_gender = df.copy()
df_gender['gender'] = ['female', 'male', 'female', 'male']
df_gender[['species', 'gender', 'course_mark']].groupby(['species', 'gender']).mean()

course_mark
species gender             
cat     female           12
human   female           82
        male             88

In [128]:
# Compute the mean course mark and concatenate the 'name' within each 'species'
df.groupby('species').agg(
    species_average=('course_mark', 'mean'), 
    names=('name', ', '.join))        # Note that we can pass in arbitrary functions for aggregation (such as ones that we define)

,species_average,names
species,,
cat,12,Lynxi
human,86,"Bonnie, Eric, Gad"


**Note**: We won't cover the equivalent of SQL HAVING, because you just do this by 
writing another row-filtering conditional statement

### Joining (i.e. SQL JOIN)

In [144]:
df_more_info = pd.DataFrame({
    'hobby': ['trivia', 'reading', 'running around at 4am', 'talking about tesla'],
    'favourite food': ['shrimp', 'brain', 'shrimp', 'oysters']},
    index=[1412, 94, 9351, 14])

df.join(df_more_info, on='student_id', how='inner')    # Note that on='student_id' wasn't necessary here, since by default it uses the index

,name,course_mark,species,hobby,favourite food
student_id,,,,,
1412,Bonnie,82,human,trivia,shrimp
94,Eric,100,human,reading,brain
9351,Lynxi,12,cat,running around at 4am,shrimp
14,Gad,76,human,talking about tesla,oysters


### Taking tops/bottom rows (i.e. SQL LIMIT)
This is useful for visualization purposes

In [140]:
# Take the top 2 rows
df.head(2)

,name,course_mark,species
student_id,,,
1412,Bonnie,82,human
94,Eric,100,human


In [141]:
# Take the bottom 2 rows
df.tail(2)

,name,course_mark,species
student_id,,,
9351,Lynxi,12,cat
14,Gad,76,human


# In-depth activity
Let's play around with Pandas on a more intricate dataset: a large survey of developers across many countries.